In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

: 

In [4]:
laptop_data=pd.read_csv("laptop_price.csv",encoding="latin-1")

In [5]:
len(laptop_data["Gpu"].unique())

110

In [74]:
#testing
def transformer(X):
    wt=[]
    ram=[]
    x_res=[]
    y_res=[]
    all_res=[]
    scrn_type=[]
    if 'kg' in X[0]:
        for i in X:
           wt.append(float(i.split("kg")[0]))
        return wt
    elif 'GB' in X[0]:
        for i in X:
             ram.append(int(i.split("GB")[0]))
        return ram
    elif re.search(r"(\d{3,4}x\d{3,4})",X[0]):
        print("Success")
        for i in X:
            temp=(re.search(r"(\d{3,4}x\d{3,4})",i).group())
            temp2=i.split(temp)
            temp=temp.split('x')
            if temp2[0]=='':
                scrn_type.append(None)
                x_res.append(int(temp[0]))
                y_res.append(int(temp[1]))
                all_res.append([None,int(temp[0]),int(temp[1])])
            else:
                scrn_type.append(temp2[0])
                x_res.append(int(temp[0]))
                y_res.append(int(temp[1]))
                all_res.append([temp2[0],int(temp[0]),int(temp[1])])
        return  all_res


In [75]:
op=transformer(to_check['ScreenResolution'])
op

Success


[['IPS Panel Retina Display ', '2560', '1600'],
 [None, '1440', '900'],
 ['Full HD ', '1920', '1080'],
 ['IPS Panel Retina Display ', '2880', '1800'],
 ['IPS Panel Retina Display ', '2560', '1600'],
 [None, '1366', '768'],
 ['IPS Panel Retina Display ', '2880', '1800'],
 [None, '1440', '900'],
 ['Full HD ', '1920', '1080'],
 ['IPS Panel Full HD ', '1920', '1080'],
 [None, '1366', '768'],
 ['Full HD ', '1920', '1080'],
 ['IPS Panel Retina Display ', '2880', '1800'],
 ['Full HD ', '1920', '1080'],
 ['IPS Panel Retina Display ', '2304', '1440'],
 ['IPS Panel Retina Display ', '2560', '1600'],
 ['Full HD ', '1920', '1080'],
 ['IPS Panel Retina Display ', '2880', '1800'],
 ['Full HD ', '1920', '1080'],
 ['IPS Panel Full HD / Touchscreen ', '1920', '1080'],
 [None, '1366', '768'],
 ['IPS Panel Full HD ', '1920', '1080'],
 [None, '1366', '768'],
 ['Full HD / Touchscreen ', '1920', '1080'],
 ['Full HD ', '1920', '1080'],
 [None, '1366', '768'],
 [None, '1440', '900'],
 ['Full HD ', '1920', '10

In [63]:
op

[[2,
  1,
  1,
  2,
  2,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  2,
  2,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  3,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,


In [51]:
laptop_data["ScreenResolution"].head(20)

0            IPS Panel Retina Display 2560x1600
1                                      1440x900
2                             Full HD 1920x1080
3            IPS Panel Retina Display 2880x1800
4            IPS Panel Retina Display 2560x1600
5                                      1366x768
6            IPS Panel Retina Display 2880x1800
7                                      1440x900
8                             Full HD 1920x1080
9                   IPS Panel Full HD 1920x1080
10                                     1366x768
11                            Full HD 1920x1080
12           IPS Panel Retina Display 2880x1800
13                            Full HD 1920x1080
14           IPS Panel Retina Display 2304x1440
15           IPS Panel Retina Display 2560x1600
16                            Full HD 1920x1080
17           IPS Panel Retina Display 2880x1800
18                            Full HD 1920x1080
19    IPS Panel Full HD / Touchscreen 1920x1080
Name: ScreenResolution, dtype: object

In [9]:
laptop_data


,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,1316,Lenovo,Yoga 500-14ISK,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,638.00
1299,1317,Lenovo,Yoga 900-13ISK,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,1499.00
1300,1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1301,1319,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00


In [ ]:
laptop_data.describe()

,laptop_ID,Inches,Price_euros
count,1303.000000,1303.000000,1303.000000
mean,660.155794,15.017191,1123.686992
std,381.172104,1.426304,699.009043
min,1.000000,10.100000,174.000000
25%,331.500000,14.000000,599.000000
50%,659.000000,15.600000,977.000000
75%,990.500000,15.600000,1487.880000
max,1320.000000,18.400000,6099.000000


In [ ]:
laptop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   object 
 8   Memory            1303 non-null   object 
 9   Gpu               1303 non-null   object 
 10  OpSys             1303 non-null   object 
 11  Weight            1303 non-null   object 
 12  Price_euros       1303 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 132.5+ KB


In [48]:
to_check=laptop_data.copy()

In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.base import BaseEstimator, TransformerMixin
# import re

# class ExtractResolution(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         pass

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         resolutions = []
#         for string in X:
#             pattern = r'\b(\d{3,4})x(\d{3,4})\b'
#             match = re.search(pattern, string)
#             if match:
#                 width = int(match.group(1))
#                 height = int(match.group(2))
#                 resolutions.append([width, height])
#             else:
#                 resolutions.append([None, None])
#         return resolutions

# # Example usage:
# data = ["IPS Panel Retina Display 2560x1600", "Another string with no resolution"]

# pipeline = Pipeline([
#     ('extract_resolution', ExtractResolution())
# ])

# result = pipeline.fit_transform(laptop_data[''])
# print(result)
